In [ ]:
"""
!pip install torch torchvision torchaudio
!pip install numpy pandas matplotlib seaborn scikit-learn
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 #for using gpu 

"""


In [2]:
import os
import torch
from torchvision import datasets,models, transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image
import torch.nn as nn
from torch.optim import Adam

In [ ]:
import pandas as pd

# read CSV
csv_path = 'image_labels_asd.csv'
df = pd.read_csv(csv_path, header=None, names=['image_name', 'label'])

# check data
print(df.head())

# Check the distribution of labels
print("distribution of labels:")
print(df['label'].value_counts())

# are there missing labels
missing_labels = df[df['label'].isnull()]
if not missing_labels.empty:
    print(f"null:\n{missing_labels}")
else:
    print("no missing label")


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, header=None, names=['image_name', 'label'])
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = int(self.annotations.iloc[idx, 1])
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

#Data Augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# DataLoader 
dataset = CustomImageDataset(csv_file='image_labels_asd.csv', img_dir='FADC-Dataset/ASD/', transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# DataLoader test 
images, labels = next(iter(train_loader))
print(f"Batch image size: {images.shape}")
print(f"Batch label: {labels}")


In [ ]:
# Create model
class EmotionDetectionModel(nn.Module):
    def __init__(self, num_classes=5):
        super(EmotionDetectionModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

model = EmotionDetectionModel(num_classes=5)
model.to('cuda')


In [ ]:
# loss funciton and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):  # 10 epoch 
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        
        # Forward Pass
        outputs = model(images)
        
        # loss 
        loss = criterion(outputs, labels)
        
        # Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Epoch [{epoch+1}/10], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


In [9]:
torch.save(model.state_dict(), 'emotion_detection_model.pth')